In [1]:
# import locale
# import os

# # Установка локали
# os.environ['LC_ALL'] = 'en_US.UTF-8'
# os.environ['LANG'] = 'en_US.UTF-8'
# locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from glob import glob
import cv2
from PIL import Image
from ultralytics import YOLO
import os

In [3]:
import kagglehub
sshikamaru_car_object_detection_path = kagglehub.dataset_download('sshikamaru/car-object-detection')

print('Data source import complete.')

Data source import complete.


In [10]:
root_dir = "/home/geraaaxx/projects/mlops_labs2_3/kaggle/working/data"
labels_dir = "/home/geraaaxx/projects/mlops_labs2_3/kaggle/working/data/labels"
images_dir = "/home/geraaaxx/projects/mlops_labs2_3/kaggle/working/data/images"

In [11]:
train_data = f"{sshikamaru_car_object_detection_path}/data/training_images"
csv_data = f"{sshikamaru_car_object_detection_path}/data/train_solution_bounding_boxes (1).csv"
test_data = f"{sshikamaru_car_object_detection_path}/data/testing_images"

In [12]:
df = pd.read_csv(csv_data)
df.head()

,image,xmin,ymin,xmax,ymax
0,vid_4_1000.jpg,281.259045,187.035071,327.727931,223.225547
1,vid_4_10000.jpg,15.163531,187.035071,120.329957,236.430180
2,vid_4_10040.jpg,239.192475,176.764801,361.968162,236.430180
3,vid_4_10020.jpg,496.483358,172.363256,630.020260,231.539575
4,vid_4_10060.jpg,16.630970,186.546010,132.558611,238.386422


In [13]:
def create_yolo_annotation(row, img_width, img_height):
    x_center = ((row['xmin'] + row['xmax']) / 2) / img_width
    y_center = ((row['ymin'] + row['ymax']) / 2) / img_height
    width = (row['xmax'] - row['xmin']) / img_width
    height = (row['ymax'] - row['ymin']) / img_height
    return f"0 {x_center} {y_center} {width} {height}"

In [14]:
for img_name in df['image'].unique():
    img_df = df[df['image'] == img_name]
    img_path = os.path.join(train_data,img_name)
    img = cv2.imread(img_path)
    if img is not None:
        img_height, img_width = img.shape[:2]

        if np.random.rand() < 0.8:
            subset = "train"
        else:
            subset = "val"

        dst_img_path = os.path.join(images_dir, subset, img_name)
        cv2.imwrite(dst_img_path, img)

        annotation_path = os.path.join(labels_dir, subset, f"{img_name.split('.')[0]}.txt")
        with open(annotation_path, 'w') as f:
            for _, row in img_df.iterrows():
                yolo_annotation = create_yolo_annotation(row, img_width, img_height)
                f.write(yolo_annotation + '\n')

In [15]:
yaml_content = f"""
path: {root_dir}
train: images/train
val: images/val

nc: 1
names: ['car']
augmentations:
  mosaic: 1.0           # Mosaic probability
  mixup: 0.2            # Mixup probability
  flipud: 0.0           # Vertical flip probability
  fliplr: 0.5           # Horizontal flip probability
  scale: 0.5            # Scale range
  shear: 0.0            # Shear angle
  perspective: 0.0      # Perspective transformation
  hsv_h: 0.015          # Hue augmentation
  hsv_s: 0.7            # Saturation augmentation
  hsv_v: 0.4            # Brightness augmentation
"""
with open('car_detection.yaml', 'w') as f:
    f.write(yaml_content)
print("YAML configuration file created.")

YAML configuration file created.
